# Importing Packages.

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [ ]:
data_dir = '/kaggle/input/cowin-vaccination-data'
registration_file = 'india_registration.csv'
vaccination_file = 'india_vaccine.csv'

# Let's Explore Registration Data First.

In [ ]:
# reading the registration file and dropping unwanted column
reg_df = pd.read_csv(os.path.join(data_dir,registration_file))
reg_df.drop('Unnamed: 0', axis=1, inplace=True)
reg_df.head()

## Little Summary about Registraion data

In [ ]:
print('dataset has {} number of features having {} records'.format(reg_df.shape[0],reg_df.shape[1]))

In [ ]:
reg_df.describe()

## Checking for missing values

In [ ]:
reg_df.isnull().sum()

## Total Registrations

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='total registrations', x=reg_df['Date'], y=reg_df['Registration_total'],marker_color='#003f5c'),
    go.Bar(name='total registrations 45+', x=reg_df['Date'], y=reg_df['Registration_45_above'],marker_color='#bc5090'),
    go.Bar(name='total registrations 18 to 45', x=reg_df['Date'], y=reg_df['Registration_18_45'],marker_color='#58508d')])
fig.update_layout(title_text='Total CoWin Registraions',barmode='group',template='plotly_white',
                  xaxis_title="date",
                  yaxis_title="count")
fig.show()

## New Registrations

In [ ]:
fig = px.bar(reg_df, x='Date', y='Registration_today', title="New Registraions Counts")
fig.update_traces(marker_color='#003f5c')
fig.update_layout(template='plotly_white',
                  xaxis_title="date",
                  yaxis_title="count")
fig.show()

## Transforming Data

In [ ]:
dates = reg_df['Date'].unique()
timestamps = []
for column in reg_df.columns:
    if len(column) == 20:
        timestamps.append(column)
values_needed = ['total','age18','age45','age60','male','female','others']
transformed_df = []
for date in dates:
    date_df = reg_df.loc[reg_df['Date'] == date]
    for timestamp in timestamps:
        timestamp_columns = [timestamp + '_' + s for s in values_needed]
        timestamp_df = date_df[timestamp_columns]
        timestamp_df.columns = values_needed
        timestamp_df['date'] = date
        timestamp_df['timestamp'] = timestamp
        transformed_df.append(timestamp_df)
transformed_df = pd.concat(transformed_df)
transformed_df.reset_index(inplace=True,drop=True)
transformed_df.head()

## New Registraions at different time intervals

In [ ]:
fig = px.bar(transformed_df, x="date", y="total", color="timestamp", title="New Registrations wrt timestamps distribution",color_discrete_sequence=px.colors.sequential.Blues)
fig.update_layout(template='plotly_white',xaxis_title="date",
                  yaxis_title="count")
fig.show()

# Favourable Time Slot

In [ ]:
idx = transformed_df.groupby(['date'])['total'].transform(max) == transformed_df['total']
most_favourable_time = transformed_df[idx].reset_index(drop=True)
print('Timestamp with max number of registrations for each day')
print(most_favourable_time[['date','timestamp','total']])
fig = px.histogram(most_favourable_time, x="timestamp",title = 'Most Favourable Time Slots with Max. Registrations')
fig.update_traces(marker_color='#003f5c')
fig.update_layout(template='plotly_white',
                  xaxis_title="time",
                  yaxis_title="count")
fig.show()

## New Registrations Distribution wrt Age, Sex

In [ ]:
datewise_grouped = transformed_df.groupby(['date'])['total', 'age18', 'age45','age60','male','female','others'].sum().reset_index()
fig = go.Figure(data=[
    go.Bar(name='new total registrations', x=datewise_grouped['date'], y=datewise_grouped['total'],marker_color='#003f5c'),
    go.Bar(name='new registrations 18+', x=datewise_grouped['date'], y=datewise_grouped['age18'],marker_color='#7a5195'),
    go.Bar(name='new registrations 45+', x=datewise_grouped['date'], y=datewise_grouped['age45'],marker_color='#ef5675'),
    go.Bar(name='new registrations 60+', x=datewise_grouped['date'], y=datewise_grouped['age60'],marker_color='#ffa600')])
fig.update_layout(title_text='New Registraions wrt Age',barmode='group',template='plotly_white',
                 xaxis_title="date",
                  yaxis_title="count")
fig.show()

fig = go.Figure(data=[
    go.Bar(name='new total registrations', x=datewise_grouped['date'], y=datewise_grouped['total'],marker_color='#003f5c'),
    go.Bar(name='new registrations male', x=datewise_grouped['date'], y=datewise_grouped['male'],marker_color='#7a5195'),
    go.Bar(name='new registrations female', x=datewise_grouped['date'], y=datewise_grouped['female'],marker_color='#ef5675'),
    go.Bar(name='new registrations others', x=datewise_grouped['date'], y=datewise_grouped['others'],marker_color='#ffa600')])
fig.update_layout(title_text='New Registraions wrt Sex',barmode='group',template='plotly_white',
                 xaxis_title="date",
                  yaxis_title="count")
fig.show()

# To Be Continued.